<div style="border: 4px groove dodgerblue; border-radius: 5px; margin-bottom: 20px;">
<h1 style="text-align: center; padding: 10px 0 15px 0;">
    <span style="color:dodgerblue;">Mask Detection</span></h1>
</div>
<img src="face.jpg" alt="Image">

Author: Ruth Yankson

<a id="content"></a>

## Table of Contents

<a href=#one>1. Importing Libraries</a>

<a href=#two>2. Loading and Preprocessing Data</a>

<a href=#three>3. Model Building</a>

<a href=#four>4. Model Training</a>

<a href=#five>5. Model Evaluation</a>

<a href=#six>6. Model Saving</a>

<a href=#seven>7. Making Predictions</a>

<a href=#eight> 8. Model Summary</a>


<a id="one"></a>

## 1. Importing Libraries

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ All the libraries used for the entire work.

---



In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import numpy as np


<a id="two"></a>

## 2. Loading and Preprocessing Data

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ Loading the train and validation directories containing the images and preprocessing each.

---



In [3]:
# Define directories
train_dir = 'Project_Images_1/data/train'
validation_dir = 'Project_Images_1/data/validation'

<div style="padding: 15px; border: 1px solid dodgerblue; border-radius: 5px; background-color: aliceblue">

<div style="padding: 15px; border-radius: 5px; background-color: white">

The **ImageDataGenerator** configuration enables various transformations to increase the diversity of the training dataset without the need to collect more data. By augmenting the dataset, the model can learn to recognize objects under different conditions, which enhances its ability to generalize to unseen data and can significantly improve its performance in real-world applications.

</div>
</div>

In [4]:

# Data augmentation and rescaling for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
# Only rescale for validation (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255)


<div style="padding: 15px; border: 1px solid dodgerblue; border-radius: 5px; background-color: aliceblue">

<div style="padding: 15px; border-radius: 5px; background-color: white">

The **flow_from_directory method** is an efficient way to manage image data for training deep learning models, especially when working with large datasets. It simplifies loading, preprocessing, and augmenting the images in real time during the training process. 

</div>
</div>

In [6]:
# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)


Found 6690 images belonging to 2 classes.


In [7]:

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary'
)

Found 864 images belonging to 2 classes.


<a id="three"></a>

## 3. Model Building

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ VGG16, a pre-trained model which works through transfer learning, is used to build a Convolutional Neural Network (CNN) model.

---



<div style="padding: 15px; border: 1px solid dodgerblue; border-radius: 5px; background-color: aliceblue">

<div style="padding: 15px; border-radius: 5px; background-color: white">

Loading the **VGG16** model with these parameters sets the stage for building a custom image classification model using transfer learning. This approach is efficient and effective, especially when working with limited datasets, as it allows the model to leverage previously learned features and adapt them to new tasks.

</div>
</div>

In [8]:
# Load the VGG16 model, excluding the top layers
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))



In [9]:
# Freeze the convolutional base
conv_base.trainable = False



<div style="padding: 15px; border: 1px solid dodgerblue; border-radius: 5px; background-color: aliceblue">

<div style="padding: 15px; border-radius: 5px; background-color: white">

**Models**, from tensorflow.keras, effectively combines the pre-trained VGG16 convolutional base with additional layers to adapt it for the specific task of mask detection. By flattening the output and adding dense layers, dropout for regularization, and a sigmoid output for binary classification, you create a robust model capable of distinguishing between individuals wearing masks and those who are not.

</div>
</div>

In [10]:
# Build the model
mask_detection_model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])


In [11]:

# Compile the model
mask_detection_model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])


<a id="four"></a>

## 4. Model Training

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ The model is trained using generators.

---



<div style="padding: 15px; border: 1px solid dodgerblue; border-radius: 5px; background-color: aliceblue">

<div style="padding: 15px; border-radius: 5px; background-color: white">

Using the fit method with the specified parameters enables the effective training of ther model on the training dataset while evaluating its performance on the validation dataset. This approach helps ensure that the model learns to generalize well to new, unseen data, a key aspect of building robust machine learning models.

</div>
</div>

In [12]:
history = mask_detection_model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)


Epoch 1/30


C:\Users\ruthy\.conda\envs\PythonReview\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 20/100 ━━━━━━━━━━━━━━━━━━━━ 6:43 5s/step - accuracy: 0.4717 - loss: 4.0934

C:\Users\ruthy\.conda\envs\PythonReview\Lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 701s 7s/step - accuracy: 0.6394 - loss: 1.9555 - val_accuracy: 0.9236 - val_loss: 0.2087
Epoch 2/30


C:\Users\ruthy\.conda\envs\PythonReview\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 691s 7s/step - accuracy: 0.8776 - loss: 0.2957 - val_accuracy: 0.9815 - val_loss: 0.1087
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 691s 7s/step - accuracy: 0.9023 - loss: 0.2469 - val_accuracy: 0.9711 - val_loss: 0.0770
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 374s 4s/step - accuracy: 0.9118 - loss: 0.2272 - val_accuracy: 0.9653 - val_loss: 0.0930
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 698s 7s/step - accuracy: 0.9063 - loss: 0.2325 - val_accuracy: 0.9769 - val_loss: 0.0685
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 693s 7s/step - accuracy: 0.9198 - loss: 0.2107 - val_accuracy: 0.9861 - val_loss: 0.0653
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 687s 7s/step - accuracy: 0.9194 - loss: 0.2034 - val_accuracy: 0.9861 - val_loss: 0.0494
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 370s 4s/step - accuracy: 0.9045 - loss: 0.2374 - val_accuracy: 0.9641 - val_loss: 0.0907
Epoch 9/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 690s 7s/step - accuracy: 0.9157 - loss: 0.2153 - val_accuracy: 0.982

<a id="five"></a>

## 5. Model Evaluation

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ Evaluating the model on the validation set.

---



In [13]:
val_loss, val_acc = mask_detection_model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc*100:.2f}%")


44/44 ━━━━━━━━━━━━━━━━━━━━ 203s 5s/step - accuracy: 0.9914 - loss: 0.0306
Validation Accuracy: 98.61%


<a id="six"></a>

## 6. Model Saving

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ Saving the model towards reusability.

---



In [14]:
mask_detection_model.save('mask_detection_model.h5')


<a id="seven"></a>

## 7. Making Predictions

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ Making predictions with neutral images.

---



In [23]:
def predict_mask_status(model, img_path):
    """
    Predicts whether a person is wearing a mask or not.

    Parameters:
    - model: The trained mask detection model.
    - img_path: The file path to the image to be evaluated.

    Returns:
    - str: A message indicating whether a mask is detected or not.
    """
    # Load the image
    img = image.load_img(img_path, target_size=(224, 224))
    
    # Convert the image to an array and preprocess
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize the image

    # Make a prediction
    prediction = model.predict(img_array)

    # Interpret the prediction
    if prediction[0] > 0.5:
        return "No Mask Detected"
    else:
        return "Mask Detected"

In [25]:
predict_mask_status(mask_detection_model, 'masktest.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


'Mask Detected'

In [26]:
predict_mask_status(mask_detection_model, 'masktest2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'No Mask Detected'

<a id="eight"></a>

## 8. Model Summary

<a href="#content">
    Back to Table of Contents
</a>

---

⚡ Summary of the model.

---



<div style="padding: 15px; border: 1px solid dodgerblue; border-radius: 5px; background-color: aliceblue">

<div style="padding: 15px; border-radius: 5px; background-color: white">

### Summary of the Mask Detection Model

The mask detection model is a deep learning-based solution designed to accurately identify whether individuals are wearing masks or not in images. Built upon the VGG16 architecture, the model leverages transfer learning to effectively adapt pre-trained weights from the ImageNet dataset for the specific task of mask detection. Below are the key aspects of the model:

1. **Architecture**:
   - The model uses the convolutional base of VGG16, a well-known convolutional neural network (CNN) architecture that excels in image classification tasks.
   - The convolutional layers of VGG16 extract relevant features from the input images, while additional layers (fully connected and dropout) are added to enable the model to classify the images into two categories: "mask" and "no mask."

2. **Data Preprocessing**:
   - The images are preprocessed using normalization (scaling pixel values to the range [0, 1]) and data augmentation techniques (such as rotation, width and height shifts, zoom, and horizontal flipping) to enhance the model's ability to generalize across various scenarios.

3. **Training Process**:
   - The model is trained using a binary cross-entropy loss function and the Adam optimizer, which efficiently updates the weights during training.
   - The training process involves using generators to yield batches of images and their corresponding labels, allowing for effective handling of large datasets.

4. **Performance**:
   - The model achieved a high validation accuracy of 98.38%, demonstrating its ability to accurately classify images of individuals with and without masks.
   - This level of accuracy indicates a strong generalization capability, making the model reliable for real-world applications.

5. **Usability**:
   - The model can be easily integrated into applications requiring mask detection, such as security systems, health monitoring, and compliance with safety regulations.
   - A reusable function has been implemented to facilitate image predictions, allowing users to quickly assess whether a person in a given image is wearing a mask.

6. **Future Enhancements**:
   - Potential enhancements include fine-tuning the model by unfreezing some layers of the convolutional base for further training on more extensive datasets.
   - Additional work can be done to improve model robustness against edge cases, such as varying angles, occlusions, and diverse lighting conditions.

### Conclusion

The mask detection model is a powerful tool for identifying mask usage in images, leveraging advanced deep learning techniques and a well-established architecture. Its high accuracy and ease of use make it suitable for deployment in various scenarios, especially in contexts where public health and safety are a priority. Future improvements could further enhance its performance and applicability in diverse environments.

</div>
</div>